In [ ]:
import os
from pathlib import Path
from groq import Groq
from pathlib import Path
import whisperx
import torch
import json
from tqdm import tqdm  # ← 這樣匯入的是函數，而非整個模組
import difflib
import re
import json
from pathlib import Path
from collections import defaultdict

base_path = Path(r"your_path")
Validation_Dataset_Formal_entity= base_path / "Validation_Dataset_Formal_entity.json"
Validation_Dataset_Formal_task2_answer = base_path / "Validation_Dataset_Formal_task2_answer.txt"

## 類別設定

In [ ]:
category_order = [
    'PATIENT', 'DOCTOR', 'USERNAME', 'PERSONALNAME', 'FAMILYNAME', 'PROFESSION',
    'ROOM', 'DEPARTMENT', 'HOSPITAL', 'ORGANIZATION', 'STREET', 'CITY', 'STATE',
    'COUNTRY', 'COUNTY', 'ZIP', 'LOCATION-OTHER', 'DISTRICT', 'AGE', 'DATE',
    'TIME', 'DURATION', 'SET', 'PHONE', 'FAX', 'EMAIL', 'URL', 'IPADDRESS',
    'OTHER', 'SOCIAL_SECURITY_NUMBER', 'MEDICAL_RECORD_NUMBER', 'HEALTH_PLAN_NUMBER',
    'ACCOUNT_NUMBER', 'LICENSE_NUMBER', 'VEHICLE_ID', 'DEVICE_ID', 'BIOMETRIC_ID',
    'ID_NUMBER'
]

In [ ]:
entity_to_categories = defaultdict(set)
category_to_entities = defaultdict(set)
entity_to_fid_category = defaultdict(list)

# 讀取 task2_answer.txt
with open(Validation_Dataset_Formal_task2_answer, encoding="utf-8") as fin:
    for line in fin:
        parts = line.strip().split("\t")
        if len(parts) >= 5:
            fid, category, _, _, entity = parts
            fid = fid.strip()
            category = category.strip()
            entity = entity.strip()
            entity_to_categories[entity].add(category)
            category_to_entities[category].add(entity)
            entity_to_fid_category[entity].append((fid, category))

# 找出同時出現在多個類別的實體
conflicted_entities = {e for e, cats in entity_to_categories.items() if len(cats) > 1}

# 印出衝突實體與其出現過的類別與 FID
print("=== ⚠️ 以下為重複出現在多個類別的實體（已排除） ===")
for entity in sorted(conflicted_entities):
    print(f"\n❗ 重複實體: \"{entity}\"")
    for fid, category in entity_to_fid_category[entity]:
        print(f"   - 出現在 FID {fid}，類別：{category}")
print(f"\n共 {len(conflicted_entities)} 筆重複實體\n")

# 確保所有類別都至少存在（即使是空集合）
for cat in category_order:
    category_to_entities.setdefault(cat, set())

# 建立結果（保留順序）
result = [
    {category: sorted([e for e in category_to_entities[category] if e not in conflicted_entities])}
    for category in category_order
]

# 輸出 JSON
with open(Validation_Dataset_Formal_entity, "w", encoding="utf-8") as fout:
    json.dump(result, fout, indent=2, ensure_ascii=False)

print(f"✅ JSON 輸出完成：{Validation_Dataset_Formal_entity}")